## Предобработка данных

In [6]:
!pip install pandas scikit-learn seaborn scipy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel

In [8]:
# Загрузка набора данных
df = pd.read_csv('heart.csv')

In [9]:
df

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,SES,CL_STATUS,MD_62,SBP_62,DBP_62,CHOL_62,WT_62,IHD_DX,DEATH
0,42,1,110,65,64,147,291,2,8,4,120,78,271,146,2,1
1,53,1,130,72,69,167,278,1,6,2,122,68,250,165,9,1
2,53,2,120,90,70,222,342,4,8,1,132,90,304,223,2,1
3,48,4,120,80,72,229,239,4,8,2,118,68,209,227,3,1
4,53,3,118,74,66,134,243,3,8,5,118,56,261,138,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,50,1,115,80,66,148,300,2,8,1,115,65,273,152,0,0
196,23,1,110,70,69,137,120,3,8,2,112,76,198,153,0,0
197,20,3,130,80,66,150,210,5,0,1,130,85,274,158,0,0
198,46,3,140,84,66,138,130,4,6,2,148,88,160,157,0,0


In [10]:
df['is_old'] = np.where(df['AGE_50'] >= 55, 1, 0)
df.is_old = df.is_old.astype('category')
df['SES'] = df['SES'].astype('category')
df['CL_STATUS'] = df['CL_STATUS'].astype('category')
df['IHD_DX'] = df['IHD_DX'].astype('category')

In [11]:
Y = df['DEATH'] # выбираем целевую переменную (категориальную)
X = df.drop('DEATH', axis=1) # переменные для проверки влияния

X = X.drop('SES', axis=1)

In [12]:
df.drop('SES', axis=1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   AGE_50     200 non-null    int64   
 1   MD_50      200 non-null    int64   
 2   SBP_50     200 non-null    int64   
 3   DBP_50     200 non-null    int64   
 4   HT_50      200 non-null    int64   
 5   WT_50      200 non-null    int64   
 6   CHOL_50    200 non-null    int64   
 7   CL_STATUS  200 non-null    category
 8   MD_62      200 non-null    int64   
 9   SBP_62     200 non-null    int64   
 10  DBP_62     200 non-null    int64   
 11  CHOL_62    200 non-null    int64   
 12  WT_62      200 non-null    int64   
 13  IHD_DX     200 non-null    category
 14  DEATH      200 non-null    int64   
 15  is_old     200 non-null    category
dtypes: category(3), int64(13)
memory usage: 21.9 KB


In [13]:
df

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,SES,CL_STATUS,MD_62,SBP_62,DBP_62,CHOL_62,WT_62,IHD_DX,DEATH,is_old
0,42,1,110,65,64,147,291,2,8,4,120,78,271,146,2,1,0
1,53,1,130,72,69,167,278,1,6,2,122,68,250,165,9,1,0
2,53,2,120,90,70,222,342,4,8,1,132,90,304,223,2,1,0
3,48,4,120,80,72,229,239,4,8,2,118,68,209,227,3,1,0
4,53,3,118,74,66,134,243,3,8,5,118,56,261,138,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,50,1,115,80,66,148,300,2,8,1,115,65,273,152,0,0,0
196,23,1,110,70,69,137,120,3,8,2,112,76,198,153,0,0,0
197,20,3,130,80,66,150,210,5,0,1,130,85,274,158,0,0,0
198,46,3,140,84,66,138,130,4,6,2,148,88,160,157,0,0,0


## Маштабирование признаков

In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   AGE_50     200 non-null    int64   
 1   MD_50      200 non-null    int64   
 2   SBP_50     200 non-null    int64   
 3   DBP_50     200 non-null    int64   
 4   HT_50      200 non-null    int64   
 5   WT_50      200 non-null    int64   
 6   CHOL_50    200 non-null    int64   
 7   CL_STATUS  200 non-null    category
 8   MD_62      200 non-null    int64   
 9   SBP_62     200 non-null    int64   
 10  DBP_62     200 non-null    int64   
 11  CHOL_62    200 non-null    int64   
 12  WT_62      200 non-null    int64   
 13  IHD_DX     200 non-null    category
 14  is_old     200 non-null    category
dtypes: category(3), int64(12)
memory usage: 20.3 KB


In [15]:
# Создаем копию данных
X_processed = X.copy()

category_columns: list[str] = X_processed.select_dtypes(include=['category']).columns # собираем колонки помеченные как category

# Применяем One-Hot Encoding
X_processed = pd.get_dummies(X_processed, columns=category_columns,drop_first=True) # drop_first=True позволяет избежать мультиколлинеарности, удаляя первый уровень категориальной переменной.


In [16]:
X_processed

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,MD_62,SBP_62,DBP_62,...,IHD_DX_1,IHD_DX_2,IHD_DX_3,IHD_DX_4,IHD_DX_5,IHD_DX_6,IHD_DX_7,IHD_DX_8,IHD_DX_9,is_old_1
0,42,1,110,65,64,147,291,4,120,78,...,False,True,False,False,False,False,False,False,False,False
1,53,1,130,72,69,167,278,2,122,68,...,False,False,False,False,False,False,False,False,True,False
2,53,2,120,90,70,222,342,1,132,90,...,False,True,False,False,False,False,False,False,False,False
3,48,4,120,80,72,229,239,2,118,68,...,False,False,True,False,False,False,False,False,False,False
4,53,3,118,74,66,134,243,5,118,56,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,50,1,115,80,66,148,300,1,115,65,...,False,False,False,False,False,False,False,False,False,False
196,23,1,110,70,69,137,120,2,112,76,...,False,False,False,False,False,False,False,False,False,False
197,20,3,130,80,66,150,210,1,130,85,...,False,False,False,False,False,False,False,False,False,False
198,46,3,140,84,66,138,130,2,148,88,...,False,False,False,False,False,False,False,False,False,False


In [17]:
# Выбираем числовые признаки
numeric_features = X_processed.select_dtypes(include=['int64']).columns.tolist()

# Инициализируем scaler
scaler = MinMaxScaler()

# Применяем нормализацию
X_processed[numeric_features] = scaler.fit_transform(X_processed[numeric_features])

In [18]:
X_processed

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,MD_62,SBP_62,DBP_62,...,IHD_DX_1,IHD_DX_2,IHD_DX_3,IHD_DX_4,IHD_DX_5,IHD_DX_6,IHD_DX_7,IHD_DX_8,IHD_DX_9,is_old_1
0,0.448980,0.000000,0.180328,0.264706,0.214286,0.279412,0.412048,0.75,0.31250,0.318841,...,False,True,False,False,False,False,False,False,False,False
1,0.673469,0.000000,0.344262,0.333333,0.571429,0.426471,0.380723,0.25,0.32500,0.173913,...,False,False,False,False,False,False,False,False,True,False
2,0.673469,0.333333,0.262295,0.509804,0.642857,0.830882,0.534940,0.00,0.38750,0.492754,...,False,True,False,False,False,False,False,False,False,False
3,0.571429,1.000000,0.262295,0.411765,0.785714,0.882353,0.286747,0.25,0.30000,0.173913,...,False,False,True,False,False,False,False,False,False,False
4,0.673469,0.666667,0.245902,0.352941,0.357143,0.183824,0.296386,1.00,0.30000,0.000000,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.612245,0.000000,0.221311,0.411765,0.357143,0.286765,0.433735,0.00,0.28125,0.130435,...,False,False,False,False,False,False,False,False,False,False
196,0.061224,0.000000,0.180328,0.313725,0.571429,0.205882,0.000000,0.25,0.26250,0.289855,...,False,False,False,False,False,False,False,False,False,False
197,0.000000,0.666667,0.344262,0.411765,0.357143,0.301471,0.216867,0.00,0.37500,0.420290,...,False,False,False,False,False,False,False,False,False,False
198,0.530612,0.666667,0.426230,0.450980,0.357143,0.213235,0.024096,0.25,0.48750,0.463768,...,False,False,False,False,False,False,False,False,False,False


### Для начала определеим функцию обучения с оценкой качества (для простоты)

In [19]:
from sklearn.model_selection import cross_val_score

def train_and_evaluate(X, Y):
    # Разделение данных на обучающую и тестовую выборки
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42, stratify=Y)

    # Инициализация модели
    model = LogisticRegression(max_iter=1000)

    # Обучение модели
    model.fit(X_train, Y_train)

    # Предсказания на обучающей выборке
    Y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(Y_train, Y_train_pred)

    # Предсказания на тестовой выборке
    Y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(Y_test, Y_test_pred)

    # Вывод результатов
    print(f"Точность на обучающей выборке: {train_accuracy:.4f}")
    print(f"Точность на тестовой выборке: {test_accuracy:.4f}")

    # Классификационный отчет
    print("\nКлассификационный отчет на тестовой выборке:")
    print(classification_report(Y_test, Y_test_pred))

    return model


In [20]:
train_and_evaluate(X_processed, Y)

Точность на обучающей выборке: 0.8063
Точность на тестовой выборке: 0.7000

Классификационный отчет на тестовой выборке:
              precision    recall  f1-score   support

           0       0.73      0.89      0.80        27
           1       0.57      0.31      0.40        13

    accuracy                           0.70        40
   macro avg       0.65      0.60      0.60        40
weighted avg       0.68      0.70      0.67        40



LogisticRegression(max_iter=1000)

### Метод 1: удаление на основе медианы

In [21]:
# Функция для удаления выбросов на основе межквартильного размаха (1.5 IQR)
def remove_outliers_based_on_iqr(X, y, iqr_multiplier=1.5):
    # Убедимся, что работаем только с числовыми данными
    X_numeric = X.select_dtypes(include=['number'])

    # Рассчитываем первый и третий квартили для каждой числовой переменной
    Q1 = X_numeric.quantile(0.25)
    Q3 = X_numeric.quantile(0.75)

    # Вычисляем межквартильный размах (IQR)
    IQR = Q3 - Q1

    # Оставляем только те значения, которые находятся в диапазоне [Q1 - 1.5*IQR, Q3 + 1.5*IQR]
    X_filtered = X_numeric[~((X_numeric < (Q1 - iqr_multiplier * IQR)) | (X_numeric > (Q3 + iqr_multiplier * IQR))).any(axis=1)]

    # Синхронизируем y с отфильтрованными X
    y_filtered = y.loc[X_filtered.index]

    return X_filtered, y_filtered


In [22]:
X_iqr, y_iqr = remove_outliers_based_on_iqr(X_processed, Y)

In [23]:
X_processed.describe()

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,MD_62,SBP_62,DBP_62,CHOL_62,WT_62
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.531939,0.453333,0.300820,0.444902,0.511429,0.434375,0.332783,0.328750,0.446750,0.417681,0.502489,0.430254
std,0.220668,0.339211,0.148481,0.107848,0.197726,0.195879,0.133793,0.299264,0.154105,0.185625,0.203099,0.199393
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.367347,0.000000,0.180328,0.392157,0.357143,0.279412,0.240964,0.000000,0.343750,0.289855,0.365721,0.295290
50%,0.551020,0.333333,0.262295,0.411765,0.500000,0.411765,0.313253,0.250000,0.425000,0.391304,0.491266,0.416667
75%,0.693878,0.666667,0.344262,0.509804,0.642857,0.588235,0.421687,0.500000,0.531250,0.492754,0.629913,0.572464
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
X_iqr.describe()

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,MD_62,SBP_62,DBP_62,CHOL_62,WT_62
count,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000,179.000000
mean,0.524228,0.450652,0.277498,0.432578,0.517159,0.423883,0.327347,0.335196,0.431599,0.402397,0.503598,0.424702
std,0.221037,0.335970,0.113995,0.081083,0.200226,0.192292,0.120857,0.298764,0.133652,0.170522,0.195940,0.195337
min,0.000000,0.000000,0.000000,0.215686,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0.048035,0.000000
25%,0.367347,0.000000,0.180328,0.392157,0.357143,0.275735,0.240964,0.000000,0.337500,0.282609,0.368996,0.286232
50%,0.551020,0.333333,0.262295,0.411765,0.571429,0.404412,0.318072,0.250000,0.406250,0.347826,0.489083,0.413043
75%,0.673469,0.666667,0.344262,0.509804,0.642857,0.577206,0.416867,0.500000,0.500000,0.492754,0.631004,0.565217
max,1.000000,1.000000,0.590164,0.656863,1.000000,0.926471,0.672289,1.000000,0.800000,0.782609,0.995633,0.891304


In [25]:
train_and_evaluate(X_iqr, y_iqr)

Точность на обучающей выборке: 0.7483
Точность на тестовой выборке: 0.6944

Классификационный отчет на тестовой выборке:
              precision    recall  f1-score   support

           0       0.71      0.96      0.81        25
           1       0.50      0.09      0.15        11

    accuracy                           0.69        36
   macro avg       0.60      0.53      0.48        36
weighted avg       0.64      0.69      0.61        36



LogisticRegression(max_iter=1000)

Несмотря на то, что точность несколько снизилась, на 0 это сказалось незначительно, но полнота в отношении 1 значительно снизилась, что может говорить о недостаточном обучении.

### Метод 2: удаление на основе среднего

In [26]:
# Функция для отбора признаков на основе среднего значения
def remove_outliers_based_on_mean(X, y, mean_multiplier=2):
    # Рассчитываем среднее значение для каждой переменной
    mean_values = X.mean()

    # Устанавливаем порог на основе среднего значения и множителя (по умолчанию 2 средних)
    threshold = mean_values.mean() * mean_multiplier

    # Отбираем признаки, у которых среднее значение выше порогового значения
    selected_features = mean_values[mean_values > threshold].index.tolist()

    # Оставляем только отобранные признаки в X
    X_filtered = X[selected_features]

    # Синхронизируем y с отфильтрованными X
    y_filtered = y.loc[X_filtered.index]

    return X_filtered, y_filtered

In [27]:
X_mean, y_mean = remove_outliers_based_on_mean(X_processed.copy(), Y.copy())

In [28]:
X_processed.describe()

,AGE_50,MD_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,MD_62,SBP_62,DBP_62,CHOL_62,WT_62
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.531939,0.453333,0.300820,0.444902,0.511429,0.434375,0.332783,0.328750,0.446750,0.417681,0.502489,0.430254
std,0.220668,0.339211,0.148481,0.107848,0.197726,0.195879,0.133793,0.299264,0.154105,0.185625,0.203099,0.199393
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.367347,0.000000,0.180328,0.392157,0.357143,0.279412,0.240964,0.000000,0.343750,0.289855,0.365721,0.295290
50%,0.551020,0.333333,0.262295,0.411765,0.500000,0.411765,0.313253,0.250000,0.425000,0.391304,0.491266,0.416667
75%,0.693878,0.666667,0.344262,0.509804,0.642857,0.588235,0.421687,0.500000,0.531250,0.492754,0.629913,0.572464
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
X_mean.describe()

,AGE_50,HT_50,CHOL_62
count,200.000000,200.000000,200.000000
mean,0.531939,0.511429,0.502489
std,0.220668,0.197726,0.203099
min,0.000000,0.000000,0.000000
25%,0.367347,0.357143,0.365721
50%,0.551020,0.500000,0.491266
75%,0.693878,0.642857,0.629913
max,1.000000,1.000000,1.000000


In [30]:
train_and_evaluate(X_mean, y_mean)

Точность на обучающей выборке: 0.7312
Точность на тестовой выборке: 0.6750

Классификационный отчет на тестовой выборке:
              precision    recall  f1-score   support

           0       0.69      0.93      0.79        27
           1       0.50      0.15      0.24        13

    accuracy                           0.68        40
   macro avg       0.60      0.54      0.51        40
weighted avg       0.63      0.68      0.61        40



LogisticRegression(max_iter=1000)

Относительно предыдущего, метода выросла точность предсказания 1, но упала точность 0, что выражается в снижении общей точности предсказаний модели.

## Методы отбора признаков

### Filter-методы

In [74]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression 

In [127]:
# Применяем SelectKBest для выбора 5 лучших признаков
selector = SelectKBest(score_func=f_regression, k=10)
X_kbest = selector.fit_transform(X_processed, Y)


In [128]:
selector.get_support(indices=True)

array([ 0,  2,  7, 10, 11, 19, 20, 23, 24, 27])

In [129]:
# Получаем список отобранных признаков
selected_features = X_processed.columns[selector.get_support(indices=True)]
print("Отобранные признаки:", selected_features)

Отобранные признаки: Index(['AGE_50', 'SBP_50', 'MD_62', 'CHOL_62', 'WT_62', 'IHD_DX_2', 'IHD_DX_3',
       'IHD_DX_6', 'IHD_DX_7', 'is_old_1'],
      dtype='object')


In [130]:
# Преобразуем X_kbest в DataFrame
X_kbest_df = pd.DataFrame(X_kbest, columns=selected_features)

# Обучаем и оцениваем модель
train_and_evaluate(X_kbest_df, Y)

Точность на обучающей выборке: 0.8000
Точность на тестовой выборке: 0.7750

Классификационный отчет на тестовой выборке:
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        27
           1       0.75      0.46      0.57        13

    accuracy                           0.78        40
   macro avg       0.77      0.69      0.71        40
weighted avg       0.77      0.78      0.76        40



LogisticRegression(max_iter=1000)

### Wrapped-методы

In [145]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Инициализируем модель логистической регрессии
base_model = LogisticRegression(max_iter=1000)

# Инициализируем RFE для выбора 5 лучших признаков
rfe = RFE(estimator=base_model, n_features_to_select=10)
rfe.fit(X_processed, Y)

# Получаем список отобранных признаков
selected_features = X_processed.columns[rfe.support_]
print("Отобранные признаки:", selected_features)

Отобранные признаки: Index(['AGE_50', 'MD_62', 'CHOL_62', 'WT_62', 'IHD_DX_1', 'IHD_DX_2',
       'IHD_DX_3', 'IHD_DX_6', 'IHD_DX_7', 'is_old_1'],
      dtype='object')


In [146]:
# Отбираем признаки
X_rfe = X_processed[selected_features]

# Обучаем и оцениваем модель
model = train_and_evaluate(X_rfe, Y)

Точность на обучающей выборке: 0.8000
Точность на тестовой выборке: 0.7750

Классификационный отчет на тестовой выборке:
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        27
           1       0.75      0.46      0.57        13

    accuracy                           0.78        40
   macro avg       0.77      0.69      0.71        40
weighted avg       0.77      0.78      0.76        40



### Embedded-методы

In [147]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

# Инициализируем модель с L1-регуляризацией
model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)

# Обучаем модель
model.fit(X_processed, Y)

# Используем SelectFromModel для отбора признаков
selector = SelectFromModel(model, prefit=True)
X_embedded = selector.transform(X_processed)

# Получаем список отобранных признаков
selected_features = X_processed.columns[selector.get_support()]
print("Отобранные признаки:", selected_features)

Отобранные признаки: Index(['AGE_50', 'MD_50', 'MD_62', 'CHOL_62', 'WT_62', 'IHD_DX_2', 'IHD_DX_3',
       'IHD_DX_6', 'IHD_DX_7', 'is_old_1'],
      dtype='object')


C:\Users\sem07\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [148]:
# Преобразуем X_embedded в DataFrame
X_embedded_df = pd.DataFrame(X_embedded, columns=selected_features)

# Обучаем и оцениваем модель
model = train_and_evaluate(X_embedded_df, Y)

Точность на обучающей выборке: 0.8000
Точность на тестовой выборке: 0.7250

Классификационный отчет на тестовой выборке:
              precision    recall  f1-score   support

           0       0.74      0.93      0.82        27
           1       0.67      0.31      0.42        13

    accuracy                           0.72        40
   macro avg       0.70      0.62      0.62        40
weighted avg       0.71      0.72      0.69        40



## Логистическая регрессия

In [40]:
import numpy as np

# Сигмоидная функция
def sigmoid(z):
    # Убедимся, что z это numpy массив
    return 1 / (1 + np.exp(-z))

# Простая реализация логистической регрессии
class OwnLogisticRegression:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.learning_rate = learning_rate
        self.iterations = iterations

    # Метод для обучения модели
    def fit(self, X, y):
        # Инициализация весов
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        # Градиентный спуск
        for _ in range(self.iterations):
            # Линейная комбинация
            linear_model = np.dot(X, self.weights) + self.bias
            # Применяем сигмоиду для предсказания
            y_pred = sigmoid(linear_model)

            # Вычисляем градиенты
            dw = (1 / len(X)) * np.dot(X.T, (y_pred - y))
            db = (1 / len(X)) * np.sum(y_pred - y)

            # Обновляем веса и смещение
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    # Метод для предсказания
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = sigmoid(linear_model)
        # Возвращаем метки классов (0 или 1)
        return [1 if i > 0.5 else 0 for i in y_pred]

In [41]:
# Пример использования
X = np.array([[0.1, 1.2], [1.1, 2.2], [2.1, 1.9], [1.0, 1.0]])
y = np.array([0, 1, 1, 0])

model = OwnLogisticRegression(learning_rate=0.1, iterations=1000)
model.fit(X, y)
predictions = model.predict(X)
print("Predictions:", predictions)

Predictions: [0, 1, 1, 0]


## Кросс-валидация

### K-fold cross-validation

In [42]:
# Импорт необходимых библиотек
import numpy as np
from sklearn.model_selection import KFold, ShuffleSplit, train_test_split, cross_val_score

In [197]:
# K-Fold Cross-Validation
def k_fold_cross_validation(X, y, n_splits=7):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []

    # Реализация K-Fold вручную
    for train_index, test_index in kf.split(X):
        model = LogisticRegression(max_iter=1000)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Обучение модели на тренировочной выборке
        model.fit(X_train, y_train)

        # Предсказания на тестовой выборке
        y_pred = model.predict(X_test)

        # Оценка точности
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    # Выводим среднюю точность и точности по каждому фолду
    print(f"K-Fold Cross-Validation - Средняя точность: {np.mean(accuracies)}")
    print(f"Точности по каждому фолду: {accuracies}")

# Пример использования:
k_fold_cross_validation(X = X_processed.to_numpy(), y = Y.to_numpy())

K-Fold Cross-Validation - Средняя точность: 0.7705840957072485
Точности по каждому фолду: [0.6896551724137931, 0.7931034482758621, 0.7241379310344828, 0.7586206896551724, 0.8214285714285714, 0.8928571428571429, 0.7142857142857143]


В моём случае 7 - оптимальное количество сплитов, оно показывает наилучший результат.

### ShuffleSplit

In [198]:
# ShuffleSplit Cross-Validation
def shuffle_split_cross_validation(X, y, n_splits=7, test_size=0.4):
    ss = ShuffleSplit(n_splits=n_splits, test_size=test_size)
    accuracies = []

    # Реализация ShuffleSplit вручную
    for train_index, test_index in ss.split(X):
        model = LogisticRegression(max_iter=1000)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Обучение модели на тренировочной выборке
        model.fit(X_train, y_train)

        # Предсказания на тестовой выборке
        y_pred = model.predict(X_test)

        # Оценка точности
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    # Выводим среднюю точность и точности по каждому разделению
    print(f"ShuffleSplit Cross-Validation - Средняя точность: {np.mean(accuracies)}")
    print(f"Точности по каждому разделению: {accuracies}")

# Пример использования:
shuffle_split_cross_validation(X_processed.to_numpy(), Y.to_numpy())


ShuffleSplit Cross-Validation - Средняя точность: 0.755357142857143
Точности по каждому разделению: [0.7, 0.6875, 0.8, 0.7875, 0.725, 0.75, 0.8375]


В моём случае Shuffle-split'а, количество n_splits не влияет на результат, он случаен в диапозоне от 73 до 77, количество n влияет лишь на возможную точность и диапозон.

### Train-Test-Validation Split

In [171]:
# Train-Test-Validation Split
def train_test_validation_split(X, y):
    model = LogisticRegression(max_iter=1000)
    # Разделяем данные на тренировочные и оставшиеся (валидация + тест)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

    # Разделяем оставшиеся данные на валидационные и тестовые
    X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Обучаем модель на тренировочных данных
    model.fit(X_train, y_train)

    # Оцениваем на валидационных данных
    y_valid_pred = model.predict(X_valid)
    validation_accuracy = accuracy_score(y_valid, y_valid_pred)
    print(f"Точность на валидационных данных: {validation_accuracy}")

    # Оцениваем на тестовых данных
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Точность на тестовых данных: {test_accuracy}")

# Пример использования:
train_test_validation_split(X_processed.to_numpy(), Y.to_numpy())


Точность на валидационных данных: 0.65
Точность на тестовых данных: 0.75


Для моего датасета прекрасно подходит кросс-валидация K-fold, так как мои данные имеют сильную взаимосвязь друг с другом. Этот метод позволет увидеть стабильную картину обучения, при моём скромном наборе данных.